In [36]:
import numpy as np

world = np.zeros((12, 12))
world[1,1:-1] = 1
world[1:-2,-2] = 1
world[1:5,1] = 1
world[5, 1:6] = 1
world[-3, 5:-1] = 1
goal = (3,3)
start = (11, 6)
world

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [37]:
# first get the mse heuristic from goal
def get_mse_heuristic(start, goal):
    heuristic = np.empty_like(world)
    for x in range(world.shape[0]):
        for y in range(world.shape[0]):
            heuristic[x,y] = (goal[0] - x)** 2 + (goal[1] - y) ** 2
    return heuristic

heuristic_matrix = get_mse_heuristic(start, goal)

In [55]:
def manhattan_distance(c1, c2):
    return np.abs(c1[0] - c2[0]) + np.abs(c1[1] - c2[1])

def get_arc_cost(c1, c2):
    if manhattan_distance(c1,c2)==1:
        cost = 1
    elif manhattan_distance(c1,c2) == 2:
        cost = 1.41
    else:
        print("Invalid")
    
    if world[c2[0],c2[1]] == 1:
        cost += 999
    
    return cost

In [56]:
# implement a* search for matrix
def a_start_search(start, goal):
    print('[goal] =', goal)

    # Empty grid
    grid = np.empty_like(world, dtype=object)
    for x in range(world.shape[1]):
        for y in range(world.shape[0]):
            grid[x,y] = {"x":x,"y":y,'label':'-',"cost":999999999999, "parent":None}

     # Set start node parameters
    grid[start[0]][start[1]]['label'] = 'S'
    grid[start[0]][start[1]]['cost'] = 0
    # Set end node parameters
    grid[goal[0]][goal[1]]['label'] = 'E'
    queue = [grid[start[0]][start[1]]]

    while queue:
        print("-------------------------------")
        queue.sort(key=lambda x:x['cost'], reverse=False) # Sort by cost
        current = queue.pop(0)                            # pop the first element
        current['label'] = 'X'

        if current==grid[goal[0]][goal[1]]:
            print('[DONE]')
            break

        neighbors = []
        if current['x']<len(world)-1:
            neighbors.append(grid[current['x']+1][current['y']]) # Right
        if current['x']>0:
            neighbors.append(grid[current['x']-1][current['y']]) # Left
        if current['y']<len(world[0])-1:
            neighbors.append(grid[current['x']][current['y']+1]) # Up
        if current['y']>0:
            neighbors.append(grid[current['x']][current['y']-1]) # Down

        print('Visited: ({0},{1})'.format(current['x'],current['y']))
        print('Neighbors:')
        for neighbor in neighbors:
            if neighbor['label'] == 'E' or neighbor['label'] == 'X':
                continue
            print('({0},{1})'.format(neighbor['x'], neighbor['y']))
            # Compute arc cost
            arc_cost = get_arc_cost((current['x'],current['y']), (neighbor['x'], neighbor['y']))
            # Compute fn=g(n)+h(n)
            heuristic_cost = heuristic_matrix[neighbor['x'], neighbor['y']]
            total_cost = arc_cost + heuristic_cost
            # Check if total cost is lower than neighbor's cost
            if total_cost < neighbor['cost']:
                # Update neighbor's parameters
                neighbor['cost'] = total_cost
                neighbor['parent'] = current
                queue.append(neighbor)
                neighbor['label'] = 'O'
            else:
                pass
                #print("Higher Arc Cost")
            
    return grid

In [57]:
grid = a_start_search(start, goal)

[goal] = (3, 3)
-------------------------------
Visited: (11,6)
Neighbors:
(10,6)
(11,7)
(11,5)
-------------------------------
Visited: (10,6)
Neighbors:
(9,6)
(10,7)
(10,5)
-------------------------------
Visited: (10,5)
Neighbors:
(11,5)
(9,5)
(10,4)
-------------------------------
Visited: (10,4)
Neighbors:
(11,4)
(9,4)
(10,3)
-------------------------------
Visited: (9,4)
Neighbors:
(8,4)
(9,5)
(9,3)
-------------------------------
Visited: (8,4)
Neighbors:
(7,4)
(8,5)
(8,3)
-------------------------------
Visited: (7,4)
Neighbors:
(6,4)
(7,5)
(7,3)
-------------------------------
Visited: (6,4)
Neighbors:
(5,4)
(6,5)
(6,3)
-------------------------------
Visited: (6,3)
Neighbors:
(7,3)
(5,3)
(6,2)
-------------------------------
Visited: (6,2)
Neighbors:
(7,2)
(5,2)
(6,1)
-------------------------------
Visited: (6,5)
Neighbors:
(7,5)
(5,5)
(6,6)
-------------------------------
Visited: (6,1)
Neighbors:
(7,1)
(5,1)
(6,0)
-------------------------------
Visited: (7,3)
Neighbors:
(

In [75]:
# get cost matrix from grid
cost_matrix = np.empty_like(world, dtype=float)
for x in range(world.shape[1]):
    for y in range(world.shape[0]):
        cost_matrix[x, y] = grid[x,y]["cost"]

# clip the cost_matrix in the range -999 to +999
cost_matrix = np.clip(cost_matrix,- 999, 999)
# display as integers
cost_matrix[goal[0], goal[1]] = -1
cost_matrix = cost_matrix.astype(int)


In [76]:
cost_matrix

array([[ 19,  14,  11,  10,  11,  14,  19,  26,  35,  46,  59,  74],
       [ 14, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999,  69],
       [ 11, 999,   3,   2,   3,   6,  11,  18,  27,  38, 999,  66],
       [ 10, 999,   2,  -1,   2,   5,  10,  17,  26,  37, 999,  65],
       [ 11, 999,   3,   2,   3,   6,  11,  18,  27,  38, 999,  66],
       [ 14, 999, 999, 999, 999, 999,  14,  21,  30,  41, 999,  69],
       [ 19,  14,  11,  10,  11,  14,  19,  26,  35,  46, 999,  74],
       [ 26,  21,  18,  17,  18,  21,  26,  33,  42,  53, 999,  81],
       [ 35,  30,  27,  26,  27,  30,  35,  42,  51,  62, 999,  90],
       [ 46,  41,  38,  37,  38, 999, 999, 999, 999, 999, 999, 101],
       [ 59,  54,  51,  50,  51,  54,  59,  66,  75,  86,  99, 114],
       [ 74,  69,  66,  65,  66,  69,   0,  81,  90, 101, 114, 129]])